<a href="https://colab.research.google.com/github/dlbkv/AI-Step/blob/master/module7/lab24_recommendation_system2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Завдання

* замініть усі пропущені значення на пусті рядки
* отримайте список з назвами фільмів
* застосуйте TF-iDF до колонки `description`
* на основі отриманих даних побудуйте таблицю з коефіцієнтами подібності
* напишіть функцію яка отримує як аргемент назву фільму та число і повертає відповідну кількість найбільш схожих фільмів

In [34]:
import pandas as pd

df = pd.read_csv("https://raw.githubusercontent.com/HalyshAnton/IT-Step-Pyton-AI/main/module7/data/netflix_titles.csv")

df.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...
3,s4,TV Show,Jailbirds New Orleans,NaN,NaN,NaN,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo..."
4,s5,TV Show,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...


In [35]:
df.fillna(value='', inplace=True)

titles = df['title']

full_descriptions = df['f_descriptions'] = df[['cast', 'listed_in', 'description']].apply(lambda row: ' '.join(row.values), axis=1)
full_descriptions

0        Documentaries As her father nears the end of ...
1       Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...
2       Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...
3        Docuseries, Reality TV Feuds, flirtations and...
4       Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...
                              ...                        
8802    Mark Ruffalo, Jake Gyllenhaal, Robert Downey J...
8803     Kids' TV, Korean TV Shows, TV Comedies While ...
8804    Jesse Eisenberg, Woody Harrelson, Emma Stone, ...
8805    Tim Allen, Courteney Cox, Chevy Chase, Kate Ma...
8806    Vicky Kaushal, Sarah-Jane Dias, Raaghav Chanan...
Length: 8807, dtype: object

In [36]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
import string
from nltk.corpus import stopwords
nltk.download("stopwords")
nltk.download('punkt')
nltk.download('wordnet')

class MyTokenizer:
     def __init__(self, stemmatize=False, lemmatize=False, stop_words=False):
         self.stemmatize = stemmatize
         self.lemmatize = lemmatize
         self.stop_words = stop_words

     def __call__(self, text):
      words = word_tokenize(text)
      if self.stop_words:
        stop_words = set(stopwords.words("english"))
        words = [word for word in words if word.lower() not in stop_words]

      words = [word for word in words if word.lower() not in string.punctuation]

      if self.stemmatize:
        stemmer = PorterStemmer()
        words = [stemmer.stem(word.lower()) for word in words]

      if self.lemmatize:
        lemmatizer = WordNetLemmatizer()
        words = [lemmatizer.lemmatize(word.lower()) for word in words]

      return words

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [37]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

tokenizer = MyTokenizer(stemmatize=True, lemmatize=True, stop_words=True)
vectorizer = TfidfVectorizer(tokenizer=tokenizer)

data = vectorizer.fit_transform(full_descriptions)

sim_matrix  = cosine_similarity(data, data)

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [38]:
df_sim = pd.DataFrame(sim_matrix,
                      columns=titles,
                      index=titles)

df_sim.head()

title,Dick Johnson Is Dead,Blood & Water,Ganglands,Jailbirds New Orleans,Kota Factory,Midnight Mass,My Little Pony: A New Generation,Sankofa,The Great British Baking Show,The Starling,...,Zak Storm,Zed Plus,Zenda,Zindagi Gulzar Hai,Zinzana,Zodiac,Zombie Dumb,Zombieland,Zoom,Zubaan
title,,,,,,,,,,,,,,,,,,,,,
Dick Johnson Is Dead,1.000000,0.000000,0.000000,0.000000,0.012906,0.000000,0.000000,0.000000,0.028992,0.037637,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.013754,0.048495
Blood & Water,0.000000,1.000000,0.022107,0.008424,0.024259,0.034111,0.013234,0.002773,0.017987,0.001666,...,0.022982,0.002866,0.030306,0.023903,0.003178,0.001664,0.053982,0.012178,0.000000,0.002956
Ganglands,0.000000,0.022107,1.000000,0.011220,0.039086,0.018633,0.029632,0.001603,0.028162,0.000000,...,0.008844,0.001656,0.001721,0.036076,0.001837,0.007504,0.054791,0.012517,0.004913,0.006418
Jailbirds New Orleans,0.000000,0.008424,0.011220,1.000000,0.044455,0.009614,0.000000,0.000000,0.048188,0.010967,...,0.004563,0.000000,0.000000,0.011364,0.000000,0.000000,0.022018,0.000000,0.011351,0.000000
Kota Factory,0.012906,0.024259,0.039086,0.044455,1.000000,0.020447,0.009433,0.001759,0.030904,0.010045,...,0.009704,0.019097,0.001888,0.057300,0.028112,0.000000,0.065299,0.017800,0.029268,0.001875


In [39]:
def get_recommendations(title, num_recommend):
  sim_titles = list(df_sim[title].nlargest(num_recommend).index)[1:]
  return sim_titles

In [40]:
title = 'Blood & Water'
k = 5

get_recommendations(title, k)

['Diamond City',
 "Kings of Jo'Burg",
 'Dive Club',
 'Elite Short Stories: Nadia Guzmán']